## Import the necessary Python libararies

In [1]:
import requests
import json
import pandas as pd
import numpy as np
import requests, os

## Read the master csv 

In [33]:
master_df = pd.read_csv("master_data.csv")

C:\Users\samsu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Explore data

print number of empty rows in budget column

In [3]:
master_df['budget'].isnull().sum()

0

print number of empty rows in revenue column

In [4]:
master_df['revenue'].isnull().sum()

3

print number of rows with budget=0

In [5]:
sum(master_df['budget']==0)

36626

print number of rows with revenue=0

In [6]:
sum(master_df['revenue']==0)

38107

## Fix missing values

create a numpy series for IMDB movie ids of all the movies for which both budget and revenue equals 0

In [80]:
missing_both = master_df['imdb_id'][(master_df['revenue']==0) & (master_df['budget']==0)]

In [57]:
missing_both.shape[0]

34588

In [9]:
missing_both.isnull().sum()

15

In [10]:
missing_both = missing_both.dropna(axis=0, how='all').values

In [11]:
missing_both

array(['tt0113228', 'tt0112302', 'tt0112896', ..., 'tt0303758',
       'tt0008536', 'tt6980792'], dtype=object)

In [12]:
len(missing_both)

34573

In [83]:
missing_both = missing_both.reset_index(drop=True)

In [84]:
movie_info = {}
fetched_missing_all = pd.DataFrame(None,None,columns=['imdb_id','budget','revenue'])

for i in range(len(missing_both)):
    budget = 0
    revenue = 0
    header = {"key":os.environ['IMDB_NET_API_KEY'],"id":missing_both[i]}
    resp = requests.post("http://imdbapi.net/api",json=header)
    if resp.status_code !=200:
        break
    rtext = json.loads(resp.text)
    #if 'status' in rtext and rtext['status']=='false':
    #    break
    if 'budget'in rtext and rtext['budget'].replace("$","").replace(",","").isdigit():
        budget = int(rtext['budget'].replace("$","").replace(",",""))
    if 'gross'in rtext and rtext['gross'].replace("$","").replace(",","").isdigit():
        revenue = int(rtext['gross'].replace("$","").replace(",",""))
    movie_info = {"imdb_id":missing_both[i],"budget":budget,"revenue":revenue}
    fetched_missing_all =  fetched_missing_all.append({'imdb_id':movie_info['imdb_id'],'budget':movie_info['budget'],'revenue':movie_info['revenue']},ignore_index=True)

0
1000
2000
3000
4000


In [111]:
fetched_missing_all.shape

(34588, 3)

In [112]:
fetched_missing_all[(fetched_missing_all['budget']!=0) & (fetched_missing_all['revenue']!=0)].count()

budget     968
imdb_id    968
revenue    968
dtype: int64

So, out of the 34573 movies that had 0 in budget and revenue columns, we were able to get both budget and revenue values for 968 movies.

Next, we pick up all the rows for which either budget or revenue has 0 value.

In [113]:
missing_one = master_df['imdb_id'][((master_df['revenue']==0) & (master_df['budget']!=0)) | ((master_df['revenue']!=0) & (master_df['budget']==0))]

In [118]:
missing_one = missing_one.dropna(axis=0, how='all').values

In [121]:
missing_one.shape[0]

5555

In [127]:
movie_info = {}
fetched_missing_one = pd.DataFrame(None,None,columns=['imdb_id','budget','revenue'])

for i in range(len(missing_one)):
    budget = 0
    revenue = 0
    header = {"key":'CwPaEy9GKVyUig6J8J2MdldyARSNOd',"id":missing_one[i]}
    resp = requests.post("http://imdbapi.net/api",json=header)
    if resp.status_code !=200:
        break
    rtext = json.loads(resp.text)
    #if 'status' in rtext and rtext['status']=='false':
    #    break
    if 'budget'in rtext and rtext['budget'].replace("$","").replace(",","").isdigit():
        budget = int(rtext['budget'].replace("$","").replace(",",""))
    if 'gross'in rtext and rtext['gross'].replace("$","").replace(",","").isdigit():
        revenue = int(rtext['gross'].replace("$","").replace(",",""))
    movie_info = {"imdb_id":missing_one[i],"budget":budget,"revenue":revenue}
    fetched_missing_one =  fetched_missing_one.append({'imdb_id':movie_info['imdb_id'],'budget':movie_info['budget'],'revenue':movie_info['revenue']},ignore_index=True)

In [128]:
fetched_missing_one.shape

(5555, 3)

In [129]:
fetched_missing_one[(fetched_missing_one['budget']!=0) & (fetched_missing_one['revenue']!=0)].count()

imdb_id    827
budget     827
revenue    827
dtype: int64

Out of 5555 movies missing either budget or revenue information, we were able to fetch information for 827 movies.

In [138]:
fetched_clean = fetched_missing_all[(fetched_missing_all['budget']!=0) & (fetched_missing_all['revenue']!=0)]

In [140]:
fetched_clean = fetched_clean.append(fetched_missing_one[(fetched_missing_one['budget']!=0) & (fetched_missing_one['revenue']!=0)], ignore_index=True)

In [142]:
fetched_clean.head()

,budget,imdb_id,revenue
0,2.5e+07,tt0113228,6.987e+07
1,3e+07,tt0112896,1.06936e+07
2,2e+07,tt0112722,3.20519e+07
3,9.5e+06,tt0114168,3.08621e+07
4,1.1e+07,tt0114057,2.11295e+06


In [143]:
fetched_clean.shape

(1795, 3)

In [144]:
#fetched_clean.to_csv("fetched_clean.csv",index=False,encoding="utf8")

In [162]:
master_df.loc[master_df.imdb_id.isin(fetched_clean.imdb_id),['budget','revenue']] = fetched_clean[['budget','revenue']]

In [166]:
master_df.to_csv('master_data_with_imputed_budget_and_revenue.csv',index=False,encoding='utf8')